In [ ]:
# Imports for Gemini (LangChain) in the same structure as OpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
# Note: get_openai_callback is specific to OpenAI and not used with Gemini

import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

# Load environment variables from .env (expects GOOGLE_API_KEY)
load_dotenv()


True

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [3]:
KEY = os.getenv("GOOGLE_API_KEY")

In [4]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.6, google_api_key=KEY)
print(llm.invoke("Reply with OK").content)

OK


In [5]:
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.6, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000283B80A3380>, default_metadata=())